In [ ]:
# Please install OpenAI SDK first: `pip3 install openai`
# !pip3 install openai

from openai import OpenAI
import json
import time

# the following is the system prompt for the task agent to explain teh interface with the terminal
interfacecontext = '''The following actions are available to you for your planning
1. `ls` - List files in the current directory
2. `cd <directory>` - Change the current directory to the specified directory
3. `cat <file>` - Display the contents of the specified file
'''

client = OpenAI(api_key="sk-9157cd146a344095baf8e3ef6454117b", base_url="https://api.deepseek.com")

def logComputationToFile(logMessage):
    # This function logs the message to the log file
    # It takes a log file name and a log message as input
    # and appends the message to the log file.
    with open("taskagent.log", 'a') as f:
        f.write(time.asctime() + " : " + logMessage + "\n")

def has_no_subtasks(text):
    return "no subtasks for this task" in text.lower()


def parseTaskList(taskList: str):
    # This function parses the task list string into a JSON format
    # and returns it as a Python dictionary.
    # It uses the json module to parse the string.
    if has_no_subtasks(taskList):
        logComputationToFile("No sub tasks for this task")
        return []
    
    logComputationToFile("parsing the tasklist")
    taskList = taskList.replace("```json", "").replace("```", "")
    data = json.loads(taskList)
    listOfTasks = data['subtasks']
    logComputationToFile("Parsed task lenghtth: " + str(len(listOfTasks)))
    for i in range(len(listOfTasks)):
        logComputationToFile("Parsed task: " + str(i) + " : " + listOfTasks[i])
    return listOfTasks


def getSubTaskList(inputTask="Say Hello"):
    # This function generates a list of sub tasks for a given task
    # returns a list of tasks if task can be broken down
    # into subtasks, else returns an empty list.
    logComputationToFile("Generating sub task list for the task: " + inputTask)
    sysRole =  "You take a task as input. " \
    "If the task can be broken down into a list of subtasks, return its list of subtasks in json format" \
    "If the task cannot be broken down into a list of subtasks, return no subtasks for this task. " \

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": sysRole},
            {"role": "user", "content": inputTask},
        ],
        stream=False
    )
    logComputationToFile("Response from the API: " + response.choices[0].message.content)
    return parseTaskList(response.choices[0].message.content)


In [11]:
print(getSubTaskList("Set up the development environment (install Node.js, npm/yarn, and a code editor)"))

['Download and install Node.js', 'Verify Node.js and npm/yarn installation', 'Choose and install a code editor (e.g., VS Code, Sublime Text, etc.)', 'Set up any necessary extensions or plugins for the code editor', 'Configure the development environment (e.g., set up project folders, initialize Git, etc.)']


In [ ]:
string = """```json
{
  "subtasks": [
    "Set up the development environment (Node.js, npm/yarn, code editor)",
    "Create a new React project using Create React App or another boilerplate",
    "Design the website layout and components",
    "Implement routing using React Router if it's a multi-page application",
    "Develop individual components (headers, footers, pages, etc.)",
    "Manage state using React hooks or state management libraries like Redux or Context API",
    "Style the components using CSS, SASS, or a CSS-in-JS library like styled-components",
    "Integrate APIs or backend services if needed",
    "Test the website for responsiveness and cross-browser compatibility",
    "Optimize performance (code splitting, lazy loading, etc.)",
    "Deploy the website to a hosting service (Netlify, Vercel, AWS, etc.)"
  ]
}
```"""
print(string)
# edit out the ```, json and ``` to make it a valid json string
string = string.replace("```json", "").replace("```", "")

jsonString = json.loads(string)
print(jsonString)

```json
{
  "subtasks": [
    "Set up the development environment (Node.js, npm/yarn, code editor)",
    "Create a new React project using Create React App or another boilerplate",
    "Design the website layout and components",
    "Implement routing using React Router if it's a multi-page application",
    "Develop individual components (headers, footers, pages, etc.)",
    "Manage state using React hooks or state management libraries like Redux or Context API",
    "Style the components using CSS, SASS, or a CSS-in-JS library like styled-components",
    "Integrate APIs or backend services if needed",
    "Test the website for responsiveness and cross-browser compatibility",
    "Optimize performance (code splitting, lazy loading, etc.)",
    "Deploy the website to a hosting service (Netlify, Vercel, AWS, etc.)"
  ]
}
```
{'subtasks': ['Set up the development environment (Node.js, npm/yarn, code editor)', 'Create a new React project using Create React App or another boilerplate', 'D

In [ ]:
class Node:
    def __init__(self, task):
        self.task = task
        # self.addSubTasks()
        self.subTasks = []  # List to store any number of subTasks, or in this case, sub tasks

    def addSubTask(self, subTaskName):
        """Add a child node with the given task"""
        child_node = Node(subTaskName)
        self.subTasks.append(child_node)
        return child_node

    def __str__(self):
        """String representation of the node"""
        return str(self.task)

class NaryTree:
    def __init__(self, root_task=None):
        self.root = Node(root_task) if root_task is not None else None

    # def preorder_traversal(self, node=None):
    #     """Pre-order traversal: root, then subTasks from left to right"""
    #     if node is None and self.root is None:
    #         return []
        
    #     if node is None:
    #         node = self.root
        
    #     result = [node.task]
    #     for child in node.subTasks:
    #         result.extend(self.preorder_traversal(child))
    #     return result

    # before visiting a parent node, we need to visit all its subTasks first
    def postorder_traversal(self, node=None):
        """Post-order traversal: subTasks from left to right, then root"""
        if node is None and self.root is None:
            return []
        
        if node is None:
            node = self.root
        
        result = []
        for child in node.subTasks:
            result.extend(self.postorder_traversal(child))
        result.append(node.task)
        return result


    # we need level order to visit all nodes and querry deepseek for subtasks of that node
    # we can use this to get subtasks of a task
    # we start with the main task, which is the root node, and the main goal of the whole operation
    # then we get the subtasks of the main task, which are the subTasks of the root node
    # then we get the subtasks of the level one subtasks, which are the subTasks of the subTasks of the root node
    # then we get subtasks of the level two subtasks, which are the subTasks of the subTasks of the subTasks of the root node
    # and so on as long as a node can be broken down into a task, or in another flavour, as long as a node has subTasks
    # def level_order_traversal(self):
    #     """Level-order traversal using a queue"""
    #     if not self.root:
    #         return []
        
    #     result = []
    #     queue = [self.root]
        
    #     while queue:
    #         current = queue.pop(0)  # Dequeue
    #         result.append(current.task)
    #         # Add all subTasks to the queue
    #         queue.extend(current.subTasks)
        
    #     return result


# Example usage
if __name__ == "__main__":
    # Create a tree
    tree = NaryTree("Go to the shop")
    
    # Level 1
    child2 = tree.root.addSubTask("Stop coding")
    child3 = tree.root.addSubTask("Walk out of the house")
    child4 = tree.root.addSubTask("Walk to the shop")
    
    # Level 2
    child2.addSubTask("Close the laptop")
    child2.addSubTask("Push chair backwards")
    child3.addSubTask("Rise up from the chair")
    child3.addSubTask("Open the door")
    child3.addSubTask("Walk out the door")
    child4.addSubTask("walk to main door")
    child4.addSubTask("Walk to the shop door")
    child4.addSubTask("Open the shop door")
    # Print traversals
    print("Pre-order traversal:", tree.preorder_traversal())
    # Output: [1, 2, 5, 6, 3, 7, 4, 8, 9, 10]
    
    print("Post-order traversal:", tree.postorder_traversal())
    # Output: [5, 6, 2, 7, 3, 8, 9, 10, 4, 1]
    
    print("Level-order traversal:", tree.level_order_traversal())
    # Output: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Pre-order traversal: ['Go to the shop', 'Stop coding', 'Close the laptop', 'Push chair backwards', 'Walk out of the house', 'Rise up from the chair', 'Open the door', 'Walk out the door', 'Walk to the shop', 'walk to main door', 'Walk to the shop door', 'Open the shop door']
Post-order traversal: ['Close the laptop', 'Push chair backwards', 'Stop coding', 'Rise up from the chair', 'Open the door', 'Walk out the door', 'Walk out of the house', 'walk to main door', 'Walk to the shop door', 'Open the shop door', 'Walk to the shop', 'Go to the shop']
Level-order traversal: ['Go to the shop', 'Stop coding', 'Walk out of the house', 'Walk to the shop', 'Close the laptop', 'Push chair backwards', 'Rise up from the chair', 'Open the door', 'Walk out the door', 'walk to main door', 'Walk to the shop door', 'Open the shop door']
